## Часть 1
Напишите скрипт, который скачивает все данные с выборов губернатора СПб 2019г для всех избирательных участков. 

Входная точка [по ссылке](http://notelections.online/region/region/st-petersburg?action=show&root=1&tvd=27820001217417&vrn=27820001217413&region=78&global=&sub_region=78&prver=0&pronetvd=null&vibid=27820001217417&type=222). Затем нужно перейти на сайты территориальных избирательных комиссий. Результаты нужно сохранить в  `cvs`-файл, `sqlite` базе данных или `parquet`-файле. В итоге должна получиться таблица с полями:
- название ТИК (территориальная избирательная комиссия, некоторое объединение избирательных участков на определенной территории)
- номер УИК (избирательный участок)
- 14 стандартных полей из итогового протокола

Скрипт должен быть в отдельном `*.py` файле или в отдельном `jupyter`-ноутбуке.

In [1]:
import csv
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
url = 'http://notelections.online/region/region/st-petersburg?action=show&root=1&tvd=27820001217417&vrn=27820001217413&region=78&global=&sub_region=78&prver=0&pronetvd=null&vibid=27820001217417&type=222'

In [3]:
columns = [['ТИК', 'УИК', 'Всего избирателей', 'Полученные бюллетени',
            'Выдано избирателям в пункте голосования', 'Выдано избирателям вне пункта голосования',
            'Погашенных бюллетеней', 'Бюллетеней в переносных ящиках', 'Бюллетеней в стационарных ящиках',
            'Недействительных бюллетеней', 'Действительных бюллетеней', 'Утраченных бюллетеней', 'Неучтенных бюллетеней',
            'Амосов', 'Беглов', 'Тихонова']]


In [4]:
html_page = requests.get(url)
page = BeautifulSoup(html_page.text, 'html.parser')
opts = page.find_all('option')[1:]
print(opts[0])

<option value="/region/st-petersburg?action=show&amp;root=178400112&amp;tvd=27820001217419&amp;vrn=27820001217413&amp;prver=0&amp;pronetvd=null&amp;region=78&amp;sub_region=78&amp;type=222&amp;vibid=27820001217419">1 Территориальная избирательная комиссия №1</option>


In [5]:
site_root = 'http://notelections.online/region'

for i in range(len(opts)):
    opt = opts[i]
    opt_parsed = BeautifulSoup(requests.get(site_root + opt.get('value')).text, 'html.parser')

    tik = opt_parsed.find_all('tr')[32:48]
    uiks = [[int(elem.text.split(' ')[-1][1:]) for elem in tik[0].find_all('a')]]

    tik.pop(-4)
    tik.pop(0)

    for data in tik:
        uiks.append([int(c.text.split('\n')[0]) for c in data.find_all('td')])

    tik_num = opt.text.split(' ')[0]

    for j in range(len(uiks[0])):
        row = [tik_num]
        for k in range(15):
            row.append(uiks[k][j])
        columns.append(row)

In [6]:
frame = pd.DataFrame(data=columns)

In [7]:
frame.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,ТИК,УИК,Всего избирателей,Полученные бюллетени,Выдано избирателям в пункте голосования,Выдано избирателям вне пункта голосования,Погашенных бюллетеней,Бюллетеней в переносных ящиках,Бюллетеней в стационарных ящиках,Недействительных бюллетеней,Действительных бюллетеней,Утраченных бюллетеней,Неучтенных бюллетеней,Амосов,Беглов,Тихонова
1,1,1,1803,1200,587,11,602,11,587,23,575,0,0,110,345,120
2,1,2,1466,1100,433,14,653,14,430,19,425,0,0,53,326,46
3,1,3,2092,1600,576,22,1002,22,576,15,583,0,0,155,332,96
4,1,4,1056,1000,318,4,678,4,318,13,309,0,0,67,171,71


In [8]:
with open('data/election_data.csv', "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerows(columns)